# Processing of the Instance Dataset

Notebook for building the Graph INSTANCE dataset from the INSTANCE dataset

 - Filter for network IV
 - Filter for earthquakes of magnitude M >= 3
 - Filter out stations which have no recordings for those earthquakes
 - Compute distances across all stations and save them in a numpy array
 - save filtered metadataframe as csv file
 - read and extract data from the large hdf5 file and create a smaller, filtered one

### Metadata Infos

 - column 'source_id' is the event ID, i.e. the earthquake identifier
 - 'station_code' is the identifier of the station like e.g. ATFO
 - 'station_network_code' is the identifier of the measurement network
 - 'station_longitude_deg' and 'station_latitude_deg' for the coordinates
 - 'station_channels' is the Seed channel description (https://scedc.caltech.edu/data/station/seed.html), like EH, HH, HN

Also potentially useful:
 - 'station_elevation_m'
 - 'source_origin_time'
 - 'source_latitude_deg' and 'source_longitude_deg'
 - 'source_depth_km'
 - 'source_magnitude'
 - 'source_type'
 - 'trace_start_time' for when the device started measuring
 - 'trace_dt_s' for measurement period

targets  in cm/s^2 and cm/s:
 - 'trace_pga_cmps2'
 - 'trace_pgv_cmps'
 - 'trace_sa03_cmps2'
 - 'trace_sa10_cmps2'
 - 'trace_sa30_cmps2'

In [9]:
import numpy as np
import pandas as pd

import warnings
from pathlib import Path

In [10]:
warnings.filterwarnings('ignore')

In [11]:
Instance_path = Path('./Instance')
GI_path = Path('./GraphInstance')

### Filter Metadata dataframe

In [29]:
events_metaData = pd.read_csv(Instance_path / 'metadata_Instance_events_v3.csv')
events_metaData.info(verbose=True)

In [30]:
# Convert argument of metadata columns with different type to numeric

list_eve = ['trace_E_min_counts', 'trace_N_min_counts', 'trace_Z_min_counts',
            'trace_E_max_counts', 'trace_N_max_counts', 'trace_Z_max_counts',
            'trace_E_median_counts', 'trace_N_median_counts', 'trace_Z_median_counts',
            'trace_E_mean_counts', 'trace_N_mean_counts', 'trace_Z_mean_counts',
            'trace_E_pga_perc', 'trace_N_pga_perc', 'trace_Z_pga_perc',
            'trace_E_pga_cmps2', 'trace_N_pga_cmps2', 'trace_Z_pga_cmps2',
            'trace_E_pgv_cmps', 'trace_N_pgv_cmps', 'trace_Z_pgv_cmps',
            'trace_E_snr_db', 'trace_N_snr_db', 'trace_Z_snr_db',
            'trace_E_sa03_cmps2', 'trace_N_sa03_cmps2', 'trace_Z_sa03_cmps2',
            'trace_pgv_cmps', 'trace_pga_perc',
            'trace_EQT_number_detections', 'trace_EQT_P_number', 'trace_EQT_S_number', 'trace_GPD_P_number',
            'trace_GPD_S_number']

for ele in list_eve:
    events_metaData[ele] = pd.to_numeric(events_metaData[ele], errors='coerce')

In [31]:
events_metaData.head(10)

In [32]:
def filter_meta_data(events_metaData, min_num_stations=10):
    # Filter for network IV
    df = events_metaData[events_metaData['station_network_code'] == 'IV']

    # Filter for magnitude >= 3.0 earthquakes
    df = df[df['source_magnitude'] >= 3.0]

    # filter out earthquakes with less than min_num_stations stations recording the event
    i = 1
    for event_id in list(df['source_id'].unique()):
        event_df = df[df['source_id'] == event_id]
        n_stations = event_df['station_code'].nunique()
        if n_stations < min_num_stations:
            # remove event
            df = df[df['source_id'] != event_id]
            i += 1

    print(f'Number of events removed as they have less than {min_num_stations} stations recording the event: {i}')

    return df


events_metaData_filtered_df = filter_meta_data(events_metaData)

In [33]:
# plot some dataset information
print(
    f"Unique sampling periods in seconds s: {events_metaData_filtered_df['trace_dt_s'].unique()}. --> It appears they are all in 100 Hz")
print(f"Number of M>=3 earthquakes measured in network IV: {events_metaData_filtered_df['source_id'].nunique()}")
print(
    f"Number of unique stations in network IV with M>=3 earthquakes: {events_metaData_filtered_df['station_code'].nunique()}")

In [34]:
# Export filtered dataframe
events_metaData_filtered_df.to_csv(GI_path / 'events_meta_data.csv', index=False)

### Create a distance matrix

In [41]:
from geopy.distance import geodesic

In [45]:
def create_distance_matrix(df):
    stations = sorted(list(df['station_code'].unique()))

    # create a longitude/latitude dictionary
    station_dict = {}
    for station in stations:
        row = df[df['station_code'] == station].iloc[0]
        lat = row['station_latitude_deg']
        lon = row['station_longitude_deg']
        station_dict[station] = {'lat': lat,
                                 'lon': lon}

    # calculate distances
    dists = np.zeros([len(stations), len(stations)])
    for i1, s1 in enumerate(stations):
        s1_lat = station_dict[s1]['lat']
        s1_lon = station_dict[s1]['lon']
        for i2, s2 in enumerate(stations):
            s2_lat = station_dict[s2]['lat']
            s2_lon = station_dict[s2]['lon']
            dists[i1, i2] = geodesic((s1_lon, s1_lat), (s2_lon, s2_lat)).km

    return dists


meta_df = pd.read_csv(GI_path / 'events_meta_data.csv')
dists = create_distance_matrix(meta_df)

In [47]:
# save distances as numpy array
np.save(GI_path / 'station_distances_km.npy', dists)

### Create stripped down HDF5 File with filtered data

This reduces the size of the HDF5 file from ~150GB to ~20GB by filtering out data which do not fit the desired criteria.

In [7]:
import h5py
import pandas as pd

In [12]:
# read metadata
meta_df = pd.read_csv(GI_path / 'events_meta_data.csv')
meta_df

In [13]:
hdf5_input_path = './../data/Instance/Instance_events_gm.hdf5'
hdf5_output_path = GI_path / 'GraphInstance_events_gm.hdf5'

In [14]:
with h5py.File(hdf5_input_path, 'r') as original_h5:
    with h5py.File(hdf5_output_path, 'w') as output_h5:
        # create 'data' group in new file to maintain the original file structure
        data_group = output_h5.create_group('data')
        for i, row in meta_df.iterrows():
            tracename = row['trace_name']
            # get waveform data
            waveform = original_h5['data'][tracename]
            # write to new file
            data_group.create_dataset(name=tracename, data=waveform)

### Splitting Procedure (Train, Validation, Test)

 - 80% training, 10% validation and 10% test split
 - The samples are shuffled to ensure there is no temporal bias (as they are otherwise ordered by date)
 - Shuffling is done stratified  wrt the magnitude and in two steps (due to limitations of sklearn). I.e. If there are 8 samples of magnitude 5, then the first split assigns 6 samples to the training split, and 2 to the temporary split. Then this temporary split is divided into 1 validation and 1 test sample.
 - Due to this approach, sklearn will raise an error if there are magnitudes with less than 8 samples, because then in the second split, it only has a single sample to divide among validation and test split.
 - Therefore, all samples with a magnitude where there are less than 8 samples in total, are filtered out. In total these are 29 samples. of magnitudes 4.9, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.8, 5.9, 6.0, 6.1, 6.5.
 - To not waste these high magnitude samples, they are again split randomly, without stratification with a 80:10:10 ratio in two steps and appended to their respective splits.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

In [ ]:
meta_df = pd.read_csv('./graph_instance_dataset_sources.csv')
meta_df.head()

In [ ]:
times = pd.to_datetime(meta_df['source_origin_time'])
magnitudes = meta_df['source_magnitude']
times.head()

In [ ]:
indices = pd.Series(np.arange(len(times)))
print(indices.head())

In [ ]:
# 1) First filter out sparse samples, i.e. ones where
magnitude_counts = magnitudes.value_counts()
sparse_magnitudes = magnitude_counts[magnitude_counts <= 7].index
# separate sparse magnitudes from the rest
sparse_mask = magnitudes.isin(sparse_magnitudes)
sparse_indices = indices[sparse_mask]
sparse_magnitudes = magnitudes[sparse_mask]
non_sparse_indices = indices[~sparse_mask]
non_sparse_magnitudes = magnitudes[~sparse_mask]

# 2) Stratified Split on the non sparse magnitude samples
# Then split the whole set into 80% training and 20% rest splits
strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, temp_idx = next(strat_split.split(non_sparse_indices, non_sparse_magnitudes))
# get the magnitudes & indices (of the whole dataset for both splits)
indices_train, indices_temp = non_sparse_indices.iloc[train_idx], non_sparse_indices.iloc[temp_idx]
# split the 20% rest into 50% validation and 50% testing
strat_split_temp = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
val_idx, test_idx = next(strat_split_temp.split(indices_temp, magnitudes.iloc[indices_temp]))
# get the indices with respect to the whole dataset
indices_val, indices_test = indices.iloc[indices_temp].iloc[val_idx], indices.iloc[indices_temp].iloc[test_idx]

# 3) Unstratified Split on the sparse magnitude samples.
sparse_train_indices, sparse_temp_indices, sparse_train_magnitudes, sparse_temp_magnitudes = train_test_split(
    sparse_indices, sparse_magnitudes, test_size=0.2, random_state=42, shuffle=True
)

sparse_val_indices, sparse_test_indices, sparse_val_magnitudes, sparse_test_magnitudes = train_test_split(
    sparse_temp_indices, sparse_temp_magnitudes, test_size=0.5, random_state=42, shuffle=True
)

# 4) Combine the non-sparse and sparse splits
train_indices = sorted(pd.concat([indices_train, sparse_train_indices]).values)
val_indices = sorted(pd.concat([indices_val, sparse_val_indices]).values)
test_indices = sorted(pd.concat([indices_test, sparse_test_indices]).values)

In [ ]:
# check for overlaps:
train_set = set(train_indices)
val_set = set(val_indices)
test_set = set(test_indices)

# Check if there is any overlap between training and validation sets
train_val_overlap = train_set.intersection(val_set)
if len(train_val_overlap) == 0:
    print("No overlap between training and validation sets.")
else:
    print(f"Overlap found between training and validation sets: {train_val_overlap}")

# Check if there is any overlap between training and test sets
train_test_overlap = train_set.intersection(test_set)
if len(train_test_overlap) == 0:
    print("No overlap between training and test sets.")
else:
    print(f"Overlap found between training and test sets: {train_test_overlap}")

# Check if there is any overlap between validation and test sets
val_test_overlap = val_set.intersection(test_set)
if len(val_test_overlap) == 0:
    print("No overlap between validation and test sets.")
else:
    print(f"Overlap found between validation and test sets: {val_test_overlap}")

In [ ]:
def split_to_txt(path, split):
    with open(path, 'w') as f:
        lines = '\n'.join([str(x) for x in split])
        f.write(lines)

In [ ]:
split_to_txt('GI_train.txt', train_indices)
split_to_txt('GI_val.txt', val_indices)
split_to_txt('GI_test.txt', test_indices)